In [26]:
import datetime
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import MetaTrader5 as mt5
import pytz
import mysql.connector
import sqlalchemy as sa
from sqlalchemy import create_engine


# ichi

In [6]:
#Calculate ichimoku and inserting to csv data:
def calc_ichimoku(d):
    tenkan_window = 9
    kijun_window = 26
    senkou_span_b_window = 52
    cloud_displacement = 26
    chikou_shift = -26
    # Tenkan-sen (Conversion Line): (9-period high + 9-period low)/2))
    nine_period_high = d['high'].rolling(window= tenkan_window).max()
    nine_period_low = d['low'].rolling(window= tenkan_window).min()
    d['tenkan_sen'] = (nine_period_high + nine_period_low) /2

    # Kijun-sen (Base Line): (26-period high + 26-period low)/2))
    period26_high = d['high'].rolling(window=kijun_window).max()
    period26_low = d['low'].rolling(window=kijun_window).min()
    d['kijun_sen'] = (period26_high + period26_low) / 2

    # Senkou Span A (Leading Span A): (Conversion Line + Base Line)/2))
    d['senkou_span_a'] = ((d['tenkan_sen'] + d['kijun_sen']) / 2).shift(cloud_displacement)

    # Senkou Span B (Leading Span B): (52-period high + 52-period low)/2))
    period52_high = d['high'].rolling(window=senkou_span_b_window).max()
    period52_low = d['low'].rolling(window=senkou_span_b_window).min()
    d['senkou_span_b'] = ((period52_high + period52_low) / 2).shift(cloud_displacement)

    # The most current closing price plotted 26 time periods behind (optional)
    d['chikou_span'] = d['close'].shift(-chikou_shift)

    #calculate cloud element:
    d['cloud_element'] = d['senkou_span_a']- d['senkou_span_b']

    return d.dropna(axis=0).set_index('time')

candles = calc_ichimoku(candles)

In [17]:
candles

,open,high,low,close,tenkan_sen,kijun_sen,senkou_span_a,senkou_span_b,chikou_span,cloud_element
time,,,,,,,,,,
2023-03-22 02:00:00,1.37084,1.37107,1.37080,1.37101,1.371090,1.371170,1.372138,1.36898,1.37118,0.003157
2023-03-22 02:10:00,1.37101,1.37135,1.37101,1.37130,1.371090,1.371170,1.372175,1.36898,1.37122,0.003195
2023-03-22 02:20:00,1.37132,1.37171,1.37127,1.37136,1.371255,1.371255,1.372138,1.36898,1.37144,0.003157
2023-03-22 02:30:00,1.37136,1.37174,1.37136,1.37161,1.371270,1.371270,1.372130,1.36898,1.37150,0.003150
2023-03-22 02:40:00,1.37161,1.37162,1.37105,1.37112,1.371270,1.371270,1.372038,1.36898,1.37116,0.003057
...,...,...,...,...,...,...,...,...,...,...
2024-01-11 10:50:00,1.33683,1.33696,1.33678,1.33678,1.336400,1.336235,1.336410,1.33703,1.33627,-0.000620
2024-01-11 11:00:00,1.33679,1.33706,1.33672,1.33697,1.336475,1.336285,1.336390,1.33703,1.33633,-0.000640
2024-01-11 11:10:00,1.33697,1.33721,1.33689,1.33698,1.336680,1.336360,1.336318,1.33703,1.33639,-0.000712


# Flat level finder

In [224]:
def level_finder(candles, threshold, element):

    levels = []
    # finding levels and their strength
    for i in range(26,len(candles[element])):

        s = 0
        while (candles[element][i] == candles[element][i-1]):
            s = s + 1
            if i == len(candles[element])-1:
                break
            else:
                i = i + 1
        if (element in ['senkou_span_a', 'senkou_span_b']) and (min(candles['low'][(i-26-s):]) < candles[element][i] < max(candles['high'][(i-26-s):])):
            continue
        elif (element in ['tenkan_sen','kijun_sen']) and (min(candles['low'][i-s:]) < candles[element][i] < max(candles['high'][i-s:])):
            continue
        else:
            level = { 'start':str(candles.index[i-s]), 'end':str(candles.index[i]), 'price':candles[element][i-1],'strength': s}
            levels.append(level)

    level_df = pd.DataFrame(levels).drop_duplicates('end').set_index('start')
    # drop levels weaker than strangth 5
    level_df = level_df[level_df.strength > threshold]
    return level_df

In [225]:
cloud_levels = level_finder(candles=candles, threshold=5, element='senkou_span_b').set_index('price')
kijun_levels = level_finder(candles=candles, threshold=5, element='kijun_sen').set_index('price')

kijun_levels = kijun_levels.drop('end', axis=1)

cloud_levels.rename(columns={'strength':'cloud_strenth'}, inplace=True)
kijun_levels.rename(columns={'strength':'kijun_strenth'}, inplace=True)

levels = pd.concat([kijun_levels,cloud_levels])

In [226]:
levels

,kijun_strenth,end,cloud_strenth
price,,,
1.091605,NaN,2023-12-01 02:40:00,7.0


In [59]:
levels.groupby(by='price', group_keys=True).apply(lambda x: x)

kijun_strenth                  end  cloud_strenth
price     price                                                       
28380.900 28380.900            NaN  2023-10-19 16:45:00           16.0
28402.900 28402.900            9.0                  NaN            NaN
28442.550 28442.550            NaN  2023-10-19 18:35:00           12.0
28993.500 28993.500            NaN  2023-10-20 10:40:00           24.0
29243.300 29243.300            6.0                  NaN            NaN
29560.500 29560.500            NaN  2023-10-21 15:35:00           10.0
31034.455 31034.455           10.0                  NaN            NaN
          31034.455            NaN  2023-10-24 01:10:00           10.0
31249.890 31249.890            NaN  2023-10-24 03:20:00           16.0
33638.250 33638.250            NaN  2023-10-28 04:40:00           10.0
33997.690 33997.690            8.0                  NaN            NaN
34295.695 34295.695            NaN  2023-11-03 17:55:00            8.0
34310.750 34310.750            8.0                  NaN            NaN
34525.500 34525.500            NaN  2023-11-04 02:55:00           13.0
35543.615 35543.615            NaN  2023-11-02 11:15:00            8.0
35692.380 35692.380            NaN  2023-11-02 09:10:00           15.0
35749.460 35749.460            7.0                  NaN            NaN

In [60]:
supper_levels = []

for price in cloud_levels.index:
    if price in kijun_levels.index:
        supper_levels.append(pd.concat([cloud_levels.loc[price], kijun_levels.loc[price]]))

pd.DataFrame(supper_levels)

,end,cloud_strenth,kijun_strenth
31034.455,2023-10-24 01:10:00,10,10


# Bermudas

In [30]:
def bermuda_clouds(candles):
    switch_index = []
    clouds = {}
    for i in range(1,len(candles['cloud_element'])):
        if candles['cloud_element'][i-1]*candles['cloud_element'][i] <= 0:
            switch_index.append(i)

    for j in range(1,len(switch_index)):
        cloud = candles[['cloud_element','senkou_span_b','senkou_span_a']][switch_index[j-1]:switch_index[j]]
        bottom = min([min(cloud['senkou_span_a']), min(cloud['senkou_span_b'])])
        top = max([max(cloud['senkou_span_a']), max(cloud['senkou_span_b'])])
        area = abs(cloud['cloud_element'].sum())
        start = cloud.index[0]
        end = cloud.index[-1]
        clouds[j] = {'cloud':cloud, 'start':start, 'end':end, 'top':top, 'bottom':bottom, 'area':area, 'bermuda':False}

    for k in range(1,len(clouds)):
        if  (clouds[k]['top']  < min(candles['low'].loc[clouds[k]['start']:])) or (clouds[k]['bottom'] > max(candles['high'].loc[clouds[k]['start']:])):
            clouds[k]['bermuda'] = True

    clouds = pd.DataFrame(clouds).transpose()
    clouds = clouds[clouds.area != 0]
    bermudas = clouds[clouds.bermuda == True]

    return bermudas

# Cloud storing

In [8]:
def cloud_flat(cloud, candles):
    time_frame = candles.index[1] - candles.index[0]
    cloud_flats = {}
    counter = 1
    s = 1
    area_under_level = cloud['cloud_element'].iloc[0].round(6)
    for i in range(1,len(cloud['senkou_span_b'])):
        if (cloud['senkou_span_b'].iloc[i].round(6) == cloud['senkou_span_b'].iloc[i-1].round(6)) and (i < len(cloud['senkou_span_b'])-1):
            s = s + 1
            area_under_level = area_under_level + cloud['cloud_element'].iloc[i].round(6)
            continue
        elif (cloud['senkou_span_b'].iloc[i].round(6) != cloud['senkou_span_b'].iloc[i-1].round(6)) and (i < len(cloud['senkou_span_b'])-1):
            start = cloud.index[i-s]
            cloud_flats[counter] = {'start':start, 'price':cloud['senkou_span_b'].iloc[i-1].round(6), 'strength':s, 'AUL':area_under_level, 'kijun_s':0}  # add   'start':cloud['time'].iloc[i-s] ,
            s = 1
            area_under_level = cloud['cloud_element'].iloc[i].round(6)
            counter = counter + 1
        elif  i == len(cloud['senkou_span_b'])-1 :
            s = s + 1
            area_under_level = area_under_level + cloud['cloud_element'].iloc[i].round(6)
            start = cloud.index[i-s]
            cloud_flats[counter] = {'start':start, 'price':cloud['senkou_span_b'].iloc[i-1].round(6), 'strength':s, 'AUL':area_under_level, 'kijun_s':0}
            counter = counter + 1
    if len(cloud['senkou_span_b']) == 1:
        start = cloud.index[0]
        cloud_flats[0] = {'start':start, 'price':cloud['senkou_span_b'].iloc[0].round(6), 'strength':1, 'AUL':area_under_level, 'kijun_s':0}

    # cheking touched levels
    for k in cloud_flats.keys():
        if  min(candles['low'][cloud_flats[k]['start']:])  <= cloud_flats[k]['price'] <= max(candles['high'][cloud_flats[k]['start']:]):
            cloud_flats[k]['touch'] = True
        else :
            cloud_flats[k]['touch'] = False

    # checking super levels
    for q in cloud_flats.keys():
        level_start = cloud_flats[q]['start']
        past_candles = candles.loc[:level_start].iloc[::-1]
        if candles['high'].loc[level_start] < cloud_flats[q]['price']:
            level_cross_past_range = past_candles[past_candles['high'] > cloud_flats[q]['price']]
            if len(level_cross_past_range) > 0:
                level_cross_past_time = level_cross_past_range.index[0]
                kijun_flat_range = candles[(candles.index > level_cross_past_time) & (candles.index < level_start) & (candles.kijun_sen.round(6) == cloud_flats[q]['price'])]
                if (level_cross_past_time - time_frame) in candles.index:
                    if (len(kijun_flat_range) > 0 ) and (candles.kijun_sen.loc[level_cross_past_time - time_frame] != cloud_flats[q]['price']):
                        kijun_flat_start = kijun_flat_range.index[0]
                        if max(candles['high'].loc[kijun_flat_start:level_start].round(6)) < cloud_flats[q]['price']:
                            cloud_flats[q]['kijun_s'] = len(kijun_flat_range)
                            cloud_flats[q]['kijun_start'] = kijun_flat_start
                            continue
                        else:
                            continue
                    else:
                        continue
                else:
                    continue
            else:
                continue

        elif candles['low'].loc[level_start] > cloud_flats[q]['price']:
            level_cross_past_range = past_candles[past_candles['low'] < cloud_flats[q]['price']]
            if len(level_cross_past_range) > 0 :
                level_cross_past_time = level_cross_past_range.index[0]
                kijun_flat_range = candles[(candles.index > level_cross_past_time) & (candles.index < level_start) & (candles.kijun_sen.round(6) == cloud_flats[q]['price'])]
                if (level_cross_past_time - time_frame) in candles.index:
                    if (len(kijun_flat_range) > 0)  and (candles.kijun_sen.loc[level_cross_past_time - time_frame] != cloud_flats[q]['price']) :
                        kijun_flat_start = kijun_flat_range.index[0]
                        if min(candles['low'].loc[kijun_flat_start:level_start]) > cloud_flats[q]['price']:
                            cloud_flats[q]['kijun_s'] = len(kijun_flat_range)
                            cloud_flats[q]['kijun_start'] = kijun_flat_start
                            continue
                        else:
                            continue
                    else:
                        continue
                else:
                    continue
            else:
                continue


    # calculating the area under each level reletively, time and counter to switch
    total_area = cloud['cloud_element'].sum().round(6)
    df = pd.DataFrame(cloud_flats).transpose()
    df['AUL_total'] = df['AUL']/total_area
    df['AUL_cum'] = df['AUL_total'].cumsum()
    df.index.name = 'level_counter'
    df['counter_to_switch'] = abs(abs(df.index - len(df)/2) - len(df)/2)
    df['time_step'] = df['strength'].cumsum().shift(1)
    df['time_step'].iloc[0] = 0
    df['step_to_switch'] = abs( abs(df['time_step'] - df['strength'].sum()/2) - df['strength'].sum()/2)



    return df

In [9]:
def clouds(candles):
    switch_index = []
    clouds = {}
    for i in range(1,len(candles['cloud_element'])):
        if candles['cloud_element'][i-1].round(6)*candles['cloud_element'][i].round(6) <= 0:
            switch_index.append(i)

    for x in range(1,len(switch_index)):
        cloud = candles[['cloud_element','senkou_span_b','senkou_span_a','kijun_sen']][switch_index[x-1]:switch_index[x]]
        bottom = min([min(cloud['senkou_span_a']), min(cloud['senkou_span_b'])])
        top = max([max(cloud['senkou_span_a']), max(cloud['senkou_span_b'])])
        area = abs(cloud['cloud_element'].sum())
        start = cloud.index[0]
        end = cloud.index[-1]
        cloud_flats = cloud_flat(cloud, candles)
 #       kijun_flats = levels_scan(cloud, 'kijun_sen', candles)

        clouds[x] = {'cloud':cloud, 'start':start, 'end':end, 'top':top, 'bottom':bottom, 'area':area, 'bermuda':False,
                     'cloud_flats':cloud_flats}    # add 'kijun_flats':kijun_flats


    for k in range(1,len(clouds)):      # bermuda checking
        if  (clouds[k]['top']  < min(candles['low'].loc[clouds[k]['start']:])) or (clouds[k]['bottom'] > max(candles['high'].loc[clouds[k]['start']:])):
            clouds[k]['bermuda'] = True

    for j in range(1,len(clouds)):      # top and bottom touched checking
        if clouds[j]['bermuda'] == True:
            clouds[j]['top_touch'] = False
            clouds[j]['bottom_touch'] = False
        else:
            if (clouds[j]['top'] < max(candles['high'].loc[clouds[j]['start']:])) and (clouds[j]['bottom'] > min(candles['low'].loc[clouds[j]['start']:])):
                clouds[j]['top_touch'] = True
                clouds[j]['bottom_touch'] = True
            elif (clouds[j]['top'] < max(candles['high'].loc[clouds[j]['start']:])) and (clouds[j]['bottom'] < min(candles['low'].loc[clouds[j]['start']:])):
                clouds[j]['top_touch'] = True
                clouds[j]['bottom_touch'] = False
            elif (clouds[j]['top'] > max(candles['high'].loc[clouds[j]['start']:])) and (clouds[j]['bottom'] > min(candles['low'].loc[clouds[j]['start']:])):
                clouds[j]['top_touch'] = False
                clouds[j]['bottom_touch'] = True

    clouds = pd.DataFrame(clouds).transpose()
#    clouds = clouds[clouds.area != 0]
#    bermudas = clouds[clouds.bermuda == True]

    return clouds


In [26]:
Clouds = clouds(candles)
Clouds

C:\Users\RED LINE\AppData\Local\Temp\ipykernel_9628\336169977.py:5: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  if candels['cloud_element'][i-1].round(6)*candels['cloud_element'][i].round(6) <= 0:
C:\Users\RED LINE\AppData\Local\Temp\ipykernel_9628\2975219073.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['time_step'].iloc[0] = 0
C:\Users\RED LINE\AppData\Local\Temp\ipykernel_9628\2975219073.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

,cloud,start,end,top,bottom,area,bermuda,cloud_flats,top_touch,bottom_touch
1,cloud_element senkou_spa...,2023-03-22 06:00:00,2023-03-22 14:20:00,1.37222,1.370162,0.03058,False,start price s...,True,True
2,cloud_element senkou_spa...,2023-03-22 14:30:00,2023-03-22 15:40:00,1.3709,1.37071,0.000635,False,start price st...,True,True
3,cloud_element senkou_spa...,2023-03-22 15:50:00,2023-03-22 15:50:00,1.370605,1.370605,0.0,False,AUL kijun_s price ...,True,True
4,cloud_element senkou_spa...,2023-03-22 16:00:00,2023-03-22 16:00:00,1.370605,1.370605,0.0,False,AUL kijun_s price ...,True,True
5,cloud_element senkou_spa...,2023-03-22 16:10:00,2023-03-22 20:10:00,1.370605,1.370267,0.004483,False,start price s...,True,True
...,...,...,...,...,...,...,...,...,...,...
1184,cloud_element senkou_spa...,2024-01-09 20:10:00,2024-01-10 05:00:00,1.339557,1.33631,0.04458,False,start price s...,True,True
1185,cloud_element senkou_spa...,2024-01-10 05:10:00,2024-01-10 12:50:00,1.33985,1.338118,0.025328,False,start price s...,False,True
1186,cloud_element senkou_spa...,2024-01-10 13:00:00,2024-01-10 14:50:00,1.338737,1.33847,0.001632,False,start price s...,True,True
1187,cloud_element senkou_spa...,2024-01-10 15:00:00,2024-01-10 22:00:00,1.33843,1.337303,0.012072,False,start price s...,True,True


In [10]:
def flat_filtering(Clouds,strength):
    filterd_flats = pd.DataFrame()
    touched_clouds = Clouds[Clouds['bermuda'] == False]
    non_zero_area = touched_clouds[touched_clouds['area'] > 0]
    for cloud_flat in non_zero_area['cloud_flats']:
        touched_flat = cloud_flat[cloud_flat['touch'] == True]
        strong_flat = touched_flat[touched_flat['strength'] >= strength]
        filterd_flats = pd.concat([filterd_flats,strong_flat])

    return filterd_flats


In [125]:
filterd_flats = flat_filtering(Clouds,2)
filterd_flats

,start,price,strength,AUL,kijun_s,touch,AUL_total,AUL_cum,counter_to_switch,time_step,step_to_switch,kijun_start
level_counter,,,,,,,,,,,,
1,2023-03-22 06:00:00,1.372215,3,-0.003032,0,True,0.09915,0.09915,1.0,0,0.0,NaN
2,2023-03-22 06:30:00,1.37222,2,-0.002055,0,True,0.067201,0.166351,2.0,3,3.0,NaN
3,2023-03-22 06:50:00,1.372205,2,-0.00187,0,True,0.061151,0.227502,3.0,5,5.0,NaN
4,2023-03-22 07:10:00,1.372175,2,-0.00187,0,True,0.061151,0.288653,4.0,7,7.0,NaN
6,2023-03-22 07:40:00,1.37181,2,-0.00155,0,True,0.050687,0.364683,6.0,10,10.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7,2024-01-10 21:40:00,1.337865,2,-0.000053,0,True,0.00439,0.999586,0.0,41,2.0,NaN
2,2024-01-10 22:20:00,1.337715,5,0.000713,0,True,0.06473,0.067181,2.0,1,1.0,NaN
3,2024-01-10 23:10:00,1.337675,8,0.003305,0,True,0.300045,0.367227,3.0,6,6.0,NaN
